University of Michigan - ROB 101 Computational Linear Algebra

## Newton-Raphson for Vector Functions

- Let $x_k$ be our current approximation of a root of the function $f$. 
- We write the linear approximation of $f$ about $x_k$ as $$f(x) \approx f(x_k) + A (x-x_k), \quad A= \frac{\partial f(x_k)}{\partial x}.$$

### Newton-Raphson for Vector Functions

- Start with an initial guess $x_0$ ($k=0$).
- Solve the linear system $A \Delta x_k = -f(x_k)$.
- Update the estimated root $x_{k+1} = x_k + \Delta x_k$.
- Repeat (go back to 2) until convergence.

In [1]:
using LinearAlgebra

function Jacobian(func, x0, h = 0.001) 
    # Numerical Jacobian of f:R^m -> R^n
    m = length(x0);  # Domain dimension
    f0 = func(x0); 
    n = length(f0); # Range dimension
    
    if m == 1 # f:R -> R^n
        return (func(x0 .+ h) .- func(x0 .- h)) ./ (2*h)
    else
        Im = Matrix(1.0I, m, m); # Create standard basis for I_m
        A = zeros(n,m); # Create Jacobian matrix
        # Compute and fill in the columns of the Jacobian using central differences
        for i = 1:m
            ei = Im[:,i:i]
            A[:,i] = (func(x0 + h*ei) - func(x0 - h*ei)) / (2*h);
        end
        return A
    end
end

Jacobian (generic function with 2 methods)

## Example 1

Find a root of $F:\mathbb{R}^4 \to \mathbb{R}^4$ near $x_0=\left[\begin{array}{cccc} -2.0 & 3.0 & \pi &-1.0\end{array} \right]^\top$ for
 
$$
F(x)=\left[\begin{array}{c}
   x_1 + 2 x_2 - x_1 (x_1 + 4 x_2) - x_2 (4 x_1 + 10 x_2) + 3 \\
 3 x_1 + 4 x_2 - x_1 (x_1 + 4 x_2) - x_2 (4 x_1 + 10 x_2) + 4  \\
                                0.5 \cos(x_1) + x_3 -\left( \sin(x_3) \right)^7  \\
                              -  2(x_2)^2  \sin(x_1) + (x_4)^3
\end{array} \right].
$$

In [2]:
function f(x) # f:R^4 -> R^4
    f1 = x[1]+2*x[2]-x[1]*(x[1]+4*x[2])-x[2]*(4*x[1]+10*x[2])+3;
    f2 = 3*x[1]+4*x[2]-x[1]*(x[1]+4*x[2])-x[2]*(4*x[1]+10*x[2])+4;
    f3 = 0.5*cos(x[1])+x[3]-sin(x[3])^7;
    f4 = -2*x[2]^2*sin(x[1])+x[4]^3;
    return [f1; f2; f3; f4]
end

# Run this for testing
@show myfunc = f;
@show x0 = rand(4,1) # initial guess, x0
println("f(x0): ", myfunc(x0))
println("A: ", Jacobian(myfunc, x0))

myfunc = f = f
x0 = rand(4, 1) = [0.2842725047144292; 0.5118962734466348; 0.05424087334106176; 0.06382970779879749]
f(x0): [0.4427319605323068, 3.0350695168544353, 0.5341738416692585, -0.14672177804768308]
A: [-3.6637151970015935 -10.512105506648162 0.0 0.0; -1.6637151970018138 -8.51210550664816 0.0 0.0; -0.14022959044224237 0.0 0.9999998222228745 0.0; -0.5030422242322391 -0.5742641339052112 0.0 0.012223694793034312]


In [3]:
myfunc = f;
x0 = [-2.; 3.; π; -1.]; # initial guess
delta = 1e-9; # set a convergence threshold
# set a max iteration so we don't get stuck in the loop forever!
MAX_ITER = 100; 
N = 1; # counter for iteration
x = x0; # root variable
# let the fun begin!
while N < MAX_ITER
    fk = myfunc(x); # evaluate f at current guess, x0
    A = Jacobian(myfunc, x); # compute Jacobian
    if abs(det(A)) < delta
        println("Newton-Raphson method did not converge; Jacobian is singular.")
        break
    else
        dx = -A \ fk; # solve the linear system
    end
    if norm(dx) < delta
        println("Converged at iteration: N = ", N)
        break
    else
        x += dx;
    end
    N += 1;
end

print("root: ", x, "\n")
print("f(root): ", myfunc(x), "\n")

Converged at iteration: N = 7
root: [-2.2595730873836657, 1.7595730873836666, 0.3180893310400119, -1.6845806986434577]
f(root): [-5.329070518200751e-15, -3.552713678800501e-15, -4.358492733391728e-17, -3.531841485937548e-10]


### [Automatic Differentiation (autodiff)](https://en.wikipedia.org/wiki/Automatic_differentiation)

- Automatic differentiation is distinct from symbolic differentiation and numerical differentiation. 
- Symbolic differentiation faces the difficulty of converting a computer program into a single mathematical expression and can lead to inefficient code. 
- Numerical differentiation (the method of finite differences) can introduce round-off errors in the discretization process and cancellation. 
- Both of these classical methods have problems with calculating higher derivatives, where complexity and errors increase.
- Finally, both of these classical methods are slow at computing partial derivatives of a function with respect to many inputs, as is needed for gradient-based optimization algorithms. 
- Automatic differentiation solves all of these problems.

In [4]:
import Pkg; Pkg.add("ForwardDiff")

using ForwardDiff

   Updating registry at `C:\Users\manig\.julia\registries\General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  99.9 %                                       ]  0.8 %>                                       ]  1.0 %3 % %                                       ]  2.4 % [==>                                      ]  2.8 %.3 %.4 %                                      ]  3.6 %                                      ]  3.8 %>                                      ]  4.1 %==>                                      ]  4.5 %4.6 % %4.9 %.3 %Fetching: [===>                                     ]  6.1 % [===>                                     ]  6.4 % [===>                                     ]  6.7 % [===>                                     ]  6.9 %.9 %                                    ]  8.7 %]  9.0 % % % %9.9 %.1 %>                                   ]  11.0 %=====>                                   ]  11.2 %>                                   ]  11.5 %12.0 %>                                  ]  12.6 %.4 %.7 %]  14.6 % [======>                             

  Resolving package versions...
  Installed libfdk_aac_jll ─────── v0.1.6+4
  Installed FFTW_jll ───────────── v3.3.9+8
  Installed x265_jll ───────────── v3.0.0+3
  Installed FreeType2_jll ──────── v2.10.1+5
  Installed libvorbis_jll ──────── v1.3.6+6
  Installed FriBidi_jll ────────── v1.0.5+6
  Installed OpenSpecFun_jll ────── v0.5.3+4
  Installed ForwardDiff ────────── v0.10.23
  Installed LAME_jll ───────────── v3.100.0+3
  Installed libpng_jll ─────────── v1.6.37+6
  Installed Opus_jll ───────────── v1.3.1+3
  Installed IntelOpenMP_jll ────── v2018.0.3+2
  Installed libass_jll ─────────── v0.14.0+4
  Installed OpenSSL_jll ────────── v1.1.1+6
  Installed Ogg_jll ────────────── v1.3.4+2
  Installed ZeroMQ_jll ─────────── v4.3.2+6
  Installed Zlib_jll ───────────── v1.2.11+18
  Installed IrrationalConstants ── v0.1.1
  Installed Bzip2_jll ──────────── v1.0.6+5
  Installed CommonSubexpressions ─ v0.3.0
  Installed Preferences ────────── v1.2.2
  Installed Compat ─────────────── v3.40

In [5]:
# Run this for testing
@show myfunc = f;
@show x0 = rand(4,1) # initial guess, x0

ForwardDiff.jacobian(myfunc, x0)

myfunc = f = f
x0 = rand(4, 1) = [0.11399515878115318; 0.42867683846002747; 0.7878765607244491; 0.6853554913361786]


4×4 Array{Float64,2}:
 -2.65741    -7.4855    0.0       0.0
 -0.657405   -5.4855    0.0       0.0
 -0.0568742   0.0       0.373594  0.0
 -0.365142   -0.195045  0.0       1.40914

In [6]:
# Compare with our finite difference Jacobian
A_ad = ForwardDiff.jacobian(myfunc, x0);
A_numerical = Jacobian(myfunc, x0)

println("Error: ", norm(A_ad - A_numerical))

Error: 1.5008753184788795e-6


![not-bad.jpg](https://i.postimg.cc/WzKBHQMM/not-bad.jpg)

**Remark:** Automatic differentiation (diving inside the method) at this time is beyond ROB 101 scope. A good course to study after ROB 101 is [Introduction to Deep Learning, STAT 157, UC Berkeley, Spring, 2019.](https://c.d2l.ai/berkeley-stat-157/)

### Newton-Raphson using Automatic Differentiation

We now implement our Newton-Raphson algorithm using the autodiff.

In [7]:
using LinearAlgebra
using ForwardDiff

function f(x) # f:R^4 -> R^4
    f1 = x[1]+2*x[2]-x[1]*(x[1]+4*x[2])-x[2]*(4*x[1]+10*x[2])+3;
    f2 = 3*x[1]+4*x[2]-x[1]*(x[1]+4*x[2])-x[2]*(4*x[1]+10*x[2])+4;
    f3 = 0.5*cos(x[1])+x[3]-sin(x[3])^7;
    f4 = -2*x[2]^2*sin(x[1])+x[4]^3;
    return [f1; f2; f3; f4]
end

f (generic function with 1 method)

In [8]:
myfunc = f;
x0 = [-2.; 3.; π; -1.]; # initial guess
delta = 1e-9; # set a convergence threshold
# set a max iteration so we don't get stuck in the loop forever!
MAX_ITER = 100; 
N = 1; # counter for iteration
x = x0; # root variable
# let the fun begin!
while N < MAX_ITER
    fk = myfunc(x); # evaluate f at current guess, x0
    A = ForwardDiff.jacobian(myfunc, x); # compute Jacobian using autodiff
    if abs(det(A)) < delta
        println("Newton-Raphson method did not converge; Jacobian is singular.")
        break
    else
        dx = -A \ fk; # solve the linear system
    end
    if norm(dx) < delta
        println("Converged at iteration: N = ", N)
        break
    else
        x += dx;
    end
    N += 1;
end

print("root: ", x, "\n")
print("f(root): ", myfunc(x), "\n")

Converged at iteration: N = 7
root: [-2.2595730873836666, 1.7595730873836664, 0.31808933104001225, -1.6845806986424827]
f(root): [1.7763568394002505e-15, 0.0, -4.5428071027142636e-17, -3.4488767397533593e-10]
